In [ ]:
! head ../tests/data/*

In [ ]:
! python -m tdf_simulator.random --output_file mysupercoolfile_random.d --config_file ../tests/data/settings.toml --num_transitions 200000    

In [ ]:
! du -sh mysupercoolfile_random.d/*

In [ ]:
import numpy as np
import timsrust_pyo3
from matplotlib import pyplot as plt

reader = timsrust_pyo3.TimsReader("mysupercoolfile_random.d")
all_frames = timsrust_pyo3.read_all_frames("mysupercoolfile_random.d")
tic = np.array([sum(x.intensities) for x in all_frames])
bpc = np.array([max(x.intensities) if x.intensities else 0 for x in all_frames])

apex = np.argmax(bpc)
time = np.array([x.rt for x in all_frames])
types = np.array([x.frame_type for x in all_frames])
left = np.searchsorted(time, all_frames[apex].rt - 10)
right = np.searchsorted(time, all_frames[apex].rt + 10)

times = time[left:right]
types = types[left:right]

ms1s = bpc[left:right][types == 0]
ms1_times = time[left:right][types == 0]
ms2s = bpc[left:right][types == 2]
ms2_times = time[left:right][types == 2]


plt.plot(ms1_times, ms1s, label="MS1")
plt.plot(ms2_times, ms2s, label="MS2")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Base peak intensity")
plt.show()

In [ ]:
frame_at_apex = all_frames[apex]
frame_at_apex

In [ ]:
def make_2d_array(x, y, z):
    out = np.zeros((x.max() + 1, y.max() + 1))
    out[x, y] = z
    return out


xs = np.array(frame_at_apex.tof_indices)
# ys = np.array(frame_at_apex.scan)

imss = []
tmp = frame_at_apex.scan_offsets
for i in range(len(tmp) - 1):
    imss.extend([i] * (tmp[i + 1] - tmp[i]))

imss = np.array(imss)

# plt.imshow(make_2d_array(xs, imss, frame_at_apex.intensities), aspect="auto")
plt.hexbin(
    reader.resolve_mzs(xs),
    imss,
    gridsize=(1000, 100),
    C=frame_at_apex.intensities,
    cmap="viridis",
)
plt.xlabel("m/z")
plt.ylabel("scan_index")

In [ ]:
max_x = xs.max()
mask = (xs > (max_x - 2000)) & (xs < (max_x + 2000))
ints = np.array(frame_at_apex.intensities)

plt.hexbin(reader.resolve_mzs(xs[mask]), imss[mask], C=ints[mask], cmap="viridis")
plt.xlabel("m/z")
plt.ylabel("scan index")
# np.unique(xs[mask], return_inverse=True)